In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
filename = list(uploaded.keys())[0]
print(filename)

In [ ]:
import pandas as pd

xls = pd.ExcelFile(filename)
print(xls.sheet_names)

In [ ]:
filename = list(uploaded.keys())[0]

# Load the sheet
df = pd.read_excel(filename, sheet_name="Jap-US", header=[0,1, 2])

df.head()

In [ ]:
# Show shape and a quick preview
print("Shape:", df.shape)
display(df.head(2))

In [ ]:
df.columns.tolist()

In [ ]:
# ============================================================
# FULL DICTIONARY OF COLUMN DEFINITIONS FOR THE NDE1.0 DATASET
# ============================================================

column_definitions = {

    # --- Building Information ---
    "Building ID": "Unique identifier of the instrumented building.",
    "B_Lat.": "Latitude of the building location (degrees).",
    "B_Long.": "Longitude of the building location (degrees).",
    "Soil info.": "Qualitative soil description (e.g., sandy clay, soft soil).",
    "Vs30": "Average shear-wave velocity in the top 30 meters (m/s). Correct unit, file mistakenly shows m/s².",
    "Height": "Total building height (cm → converted to meters).",
    "No. of story": "Number of above-ground stories.",
    "Typology": "Structural system type (e.g., RC, SRC, Steel).",

    # --- Earthquake/Event Information ---
    "Event Date": "Date and time of the recorded earthquake event.",
    "E_Lat.": "Earthquake epicenter latitude (degrees).",
    "E_Long.": "Earthquake epicenter longitude (degrees).",
    "Magnitude": "Earthquake moment magnitude (Mw).",
    "Epicentral distance (R)": "Distance from building to earthquake epicenter (km).",

    # --- Ordinate Intensity Measures ---
    "PGA": "Peak Ground Acceleration (cm/s² → converted to g).",
    "PGV": "Peak Ground Velocity (cm/s → converted to m/s).",
    "PGD": "Peak Ground Displacement (cm → meters).",
    "AI": "Arias Intensity, an energy-based measure defined by ∫ a(t)^2 dt (cm/s).",
    "DP": "Duration parameter (cm·s), related to cumulative shaking effects.",
    "CAV": "Cumulative Absolute Velocity (∫ |a(t)| dt) measured in cm/s.",

    # --- Spectral Intensity Measures ---
    "SA1": "Spectral acceleration at first spectral period band (cm/s² → g).",
    "SV1": "Spectral velocity at first period (cm/s).",
    "SD1": "Spectral displacement at first period (cm).",
    "SA2": "Spectral acceleration at second period band (cm/s² → g).",
    "SV2": "Spectral velocity at second period (cm/s).",
    "SD2": "Spectral displacement at second period (cm).",

    # --- Building Response (Output Quantities, Not Inputs) ---
    "Drift": "Maximum interstory drift ratio (dimensionless, cm/cm).",
    "PTA": "Peak Top Acceleration measured at building top (cm/s²).",
    "PTV": "Peak Top Velocity measured at building top (cm/s).",
    "PTD": "Peak Top Displacement measured at building top (cm).",

    # --- Building Frequency ---
    "F1": "Fundamental (first-mode) frequency of the building (Hz).",
    "F2": "Second-mode frequency of the building (Hz).",

    # --- Average Spectral Values ---
    "Avg_Sa": "Average spectral acceleration over specified period range (cm/s²).",
    "Avg_Sv": "Average spectral velocity (cm/s).",
    "Avg_Sd": "Average spectral displacement (cm).",

    # --- Duration of Strong Motion ---
    "Effective": "Effective duration of strong motion (seconds).",
    "Bracketed 1 [0.05g]": "Bracketed duration between first and last exceedance of 0.05 g.",
    "Uniform [0.05g]": "Continuous duration above 0.05 g.",
    "Bracketed [0.1g]": "Bracketed duration above acceleration threshold 0.1 g.",
    "Uniform [0.1g]": "Continuous duration above 0.1 g.",
    "Bracketed [0.15g]": "Bracketed duration above 0.15 g.",
    "Uniform [0.15g]": "Continuous duration above 0.15 g.",
    "Bracketed [0.2g]": "Bracketed duration above 0.2 g.",
    "Uniform [0.2g]": "Continuous duration above 0.2 g.",
    "Significant_a1 [(5-75)%]": "Significant duration based on Arias Intensity (5–75%).",
    "Significant_a2 [(5-95)%]": "Significant duration based on Arias Intensity (5–95%).",
    "Significant_v1 [(5-75)%]": "Velocity-based significant duration (5–75%).",
    "Significant_v2 [(5-95)%]": "Velocity-based significant duration (5–95%).",
    "Significant_d1 [(5-75)%]": "Displacement-based significant duration (5–75%).",
    "Significant_d2 [(5-95)%]": "Displacement-based significant duration (5–95%).",
    "ZX": "Additional duration/intensity metric defined in the dataset documentation (seconds)."
}

print("Loaded dictionary with", len(column_definitions), "column definitions.")

In [ ]:
import pandas as pd

# Load the dataset with 3 header rows
df_raw = pd.read_excel(filename, sheet_name="Jap-US", header=[0,1,2])

# Build clean single-level column names
clean_names = []
for c in df_raw.columns:
    category, name, unit = c
    clean = f"{name}".strip().replace(" ", "_")
    clean_names.append(clean)

df = df_raw.copy()
df.columns = clean_names

# ---- SELECT RELEVANT COLUMNS ----
keep = [
    "Vs30",
    "Height",
    "No._of_story",
    "Typology",
    "PGA",
    "PGV",
    "PGD",
    "AI",
    "SA1",
    "SA2",
    "Drift",
    "PTA",
    "PTV",
    "PTD",
    "F1",
    "F2",
    "Effective",
    "Significant_a2_[(5-95)%]"
]

df = df[keep]

# ---- UNIT CONVERSIONS ----

# Convert height (cm → m)
df["Height"] = df["Height"] / 100.0

# Vs30: fix units in file (reported as m/s^2, but intended to be m/s)
# No conversion needed
df["Vs30"] = pd.to_numeric(df["Vs30"], errors="coerce")

# Convert PGA, SA1, SA2, PTA (cm/s^2 → g)
to_g = ["PGA", "SA1", "SA2", "PTA"]
for col in to_g:
    df[col] = df[col] / 981.0

# Convert PGV, PTV (cm/s → m/s)
for col in ["PGV", "PTV"]:
    df[col] = df[col] / 100.0

# Convert PGD, PTD (cm → m)
for col in ["PGD", "PTD"]:
    df[col] = df[col] / 100.0

# Drift is already dimensionless; F1, F2 are Hz; durations already sec.

# ---- CLEAN TYPLOGY ----
df["Typology"] = df["Typology"].astype("category")

# ---- REMOVE ROWS WITH MISSING TARGET ----
df = df.dropna(subset=["Drift"])

print("Final dataset shape:", df.shape)
df.head()

In [ ]:
df.info()

In [ ]:
import numpy as np

# Create performance level from drift ratio
def performance_level(drift):
    if drift < 0.005:
        return "IO"
    elif drift < 0.02:
        return "LS"
    else:
        return "CP"

df["PerfLevel"] = df["Drift"].apply(performance_level)

df["PerfLevel"].value_counts()

In [ ]:
# Dataset skewed towards one performance level
# Classifiers dropped

# Impute missing values (already done for classification, but repeat for safety)
df["Vs30"] = df["Vs30"].fillna(df["Vs30"].median())
df["Effective"] = df["Effective"].fillna(df["Effective"].median())
df["F1"] = df["F1"].fillna(df["F1"].median())

# One-hot encode Typology, drop_first to avoid dummy trap
df_reg = pd.get_dummies(df, columns=["Typology"], drop_first=True)

# Define features and target
X = df_reg.drop(columns=["Drift"])
y = df_reg["Drift"]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train.shape, X_test.shape

In [ ]:
#  Clean X_train / X_test so they're fully numeric -----
# If you still have PerfLevel or other string columns, drop them.
cols_to_drop = ["PerfLevel", "Building_ID", "Network", "Event_Date", "Event ID"]
for c in cols_to_drop:
    if c in X_train.columns:
        X_train = X_train.drop(columns=[c])
        X_test  = X_test.drop(columns=[c])

# If any non-numeric columns remain, drop or encode them:
non_numeric = [c for c,d in X_train.dtypes.items() if d == "object"]
print("Non-numeric columns still present (will be dropped):", non_numeric)
X_train = X_train.drop(columns=non_numeric)
X_test  = X_test.drop(columns=non_numeric)

#  Ensure index alignment after drops -----
X_train, X_test = X_train.align(X_test, join='inner', axis=1)  # keep only common columns

# Standardize features for Linear Regression (helps numerics) -----
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# Retrain models -----
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

rf = RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)   # RF can take unscaled features; we use original X_train

In [ ]:
# Evaluate -----
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

def evaluate(model, name, Xeval, Yeval):
    y_pred = model.predict(Xeval)
    mae = mean_absolute_error(Yeval, y_pred)
    rmse = np.sqrt(mean_squared_error(Yeval, y_pred))
    r2 = r2_score(Yeval, y_pred)
    print(f"\n{name} Performance:")
    print(f"MAE:  {mae:.6f}")
    print(f"RMSE: {rmse:.6f}")
    print(f"R²:   {r2:.6f}")
    return y_pred

# Evaluate Linear Regression on scaled data
y_pred_lin = evaluate(linreg, "Linear Regression", X_test_scaled, y_test)

# Evaluate RF on original (unscaled) data
y_pred_rf  = evaluate(rf, "Random Forest", X_test, y_test)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(10,6))
plt.title("Feature Importance Ranking (Random Forest)")
plt.bar(range(len(importances)), importances[indices])
plt.xticks(range(len(importances)), X.columns[indices], rotation=90)
plt.tight_layout()
plt.show()

# Print top 10
for i in indices[:10]:
    print(f"{X.columns[i]}: {importances[i]:.4f}")

In [ ]:
# Outout variables dropped


# Defensive re-build and train/eval block
import pandas as pd
import numpy as np

# 1) Recreate df_reg from df to ensure we start from the same known dataframe
# (df is the cleaned dataframe you created earlier)
df_reg = pd.get_dummies(df, columns=["Typology"], drop_first=True)

# 2) Define target and full candidate feature set, then remove response columns & metadata
target = "Drift"

# Columns that are responses that must NOT be used as predictors
bad_cols = [target, "PTA", "PTV", "PTD", "PerfLevel", "Building ID", "Network", "Event Date", "Event ID"]
# drop any that don't exist safely
cols_to_drop = [c for c in bad_cols if c in df_reg.columns]

X = df_reg.drop(columns=cols_to_drop, errors='ignore')
y = df_reg[target].astype(float)

# 3) Defensive check: list any non-numeric columns remaining
non_numeric = X.select_dtypes(include=['object','category']).columns.tolist()
print("Non-numeric columns found (should be none):", non_numeric)

# If any non-numeric remain, either encode or drop them; here we will one-hot encode object cols if present
if len(non_numeric) > 0:
    X = pd.get_dummies(X, columns=non_numeric, drop_first=True)
    print("One-hot encoded the remaining non-numeric columns.")

# Final numeric check
non_numeric_after = X.select_dtypes(include=['object','category']).columns.tolist()
print("Non-numeric columns after encoding (should be empty):", non_numeric_after)

In [ ]:
# 4) Train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 5) Scale numeric features for linear models
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# 6) Train Linear Regression and Random Forest
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

rf = RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

In [ ]:
# 7) Evaluate helper
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
def evaluate(model, name, Xeval, Yeval):
    y_pred = model.predict(Xeval)
    mae = mean_absolute_error(Yeval, y_pred)
    rmse = np.sqrt(mean_squared_error(Yeval, y_pred))
    r2 = r2_score(Yeval, y_pred)
    print(f"\n{name} Performance:")
    print(f"MAE:  {mae:.6e}")
    print(f"RMSE: {rmse:.6e}")
    print(f"R²:   {r2:.6f}")
    return y_pred

y_pred_lin = evaluate(linreg, "Linear Regression", X_test_scaled, y_test)
y_pred_rf  = evaluate(rf, "Random Forest", X_test, y_test)

In [ ]:
# 8) Feature importances from Random Forest (show top 15)
import matplotlib.pyplot as plt
importances = rf.feature_importances_
feat_names = X.columns
indices = np.argsort(importances)[::-1]

topn = 15
plt.figure(figsize=(10,6))
plt.title("Top feature importances (Random Forest)")
plt.barh(range(topn), importances[indices][:topn][::-1])
plt.yticks(range(topn), feat_names[indices][:topn][::-1])
plt.xlabel("Importance")
plt.tight_layout()
plt.savefig("feature_importance.png", dpi=300, bbox_inches='tight')
plt.show()

print("\nTop features:")
for i in indices[:topn]:
    print(f"{feat_names[i]}: {importances[i]:.4f}")

In [ ]:
import pandas as pd

coef_table = pd.DataFrame({
    "Feature": X_train.columns,
    "Coefficient": linreg.coef_
})
coef_table
